In [ ]:
# default_exp view_seg

In [ ]:
# export
import openslide
from pathlib import Path
import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import PIL.Image as Image

In [ ]:
# export
SEG_DATA = Path('/data/DeepLearning/SCHWOB_Robin/Seg_outputs_iter_3')
IMG_DATA = Path('/data/DeepLearning/SCHWOB_Robin/Patches_normalized_299/Patches_normalized_299_1')
GRADE = '1'
SCAN = 'CF_PACS05HE_15008-33373'
ITER = 3
SCAN_PATH = IMG_DATA/GRADE/SCAN
_SEGMENT_CSV = SEG_DATA/'Segmentation'/f'iter_{ITER}'/GRADE/f'{SCAN}.csv'
SEGMENT_CSV = SEG_DATA/'Segmentation'/GRADE/f'{SCAN}.csv'
_GRAPH_PATH = SEG_DATA/f'Tree/iter_{ITER}/most_significant_tree.p'
GRAPH_PATH = SEG_DATA/f'Tree_1/most_significant_tree_final.p'
_SEGMENT_STACK = SEG_DATA/f'Segmentation/iter_{ITER}/segment_stack.csv'
SEGMENT_STACK = SEG_DATA/f'Segmentation/segment_stack.csv'
OUTFOLDER = IMG_DATA/'Segmentation'

In [ ]:
# export
parser = ArgumentParser()
parser.add_argument('--file', '-f', help='notebook convenience')
parser.add_argument('--HistoryManager.hist_file', help='nbconvert convenience')
parser.add_argument('--seg-data', default=SEG_DATA, type=Path, help='Path to clustering output folder')
parser.add_argument('--img-data', default=IMG_DATA, type=Path, help='Path to image folder')
parser.add_argument('--iter', default=ITER, type=int, help='Iteration to load clustering info from')
parser.add_argument('--outfolder', default=OUTFOLDER, type=Path, help='Folder to store segments in')

_StoreAction(option_strings=['--outfolder'], dest='outfolder', nargs=None, const=None, default=PosixPath('/data/DeepLearning/SCHWOB_Robin/Patches_normalized_299/Patches_normalized_299_1/Segmentation'), type=<class 'pathlib.Path'>, choices=None, help='Folder to store segments in', metavar=None)

In [ ]:
# export
args = parser.parse_args()

In [ ]:
# export
segment_stack = args.seg_data/f'Segmentation/segment_stack.csv'
df = pd.read_csv(segment_stack)

In [ ]:
# export
graph_path = args.seg_data/f'Tree_1/most_significant_tree_final.p'
with graph_path.open('rb') as f:
    tree = pickle.load(f)

In [ ]:
# export
concepts = [10551, 10554, 12192, 12251, 12615, 10669, 10682, 14543]

In [ ]:
# export
centroids = np.stack([tree.descriptor[k] for k in concepts])

In [ ]:
# export
classes = {'K': [10551], 'K_impur': [10554, 12192], 'Sain': [12251, 12615, 10669], 'Sain_impur': [10682], 'Trash': [14543]}
concept2classes = {conc: cl for cl in classes for conc in classes[cl]}

In [ ]:
# export
class2color = {'K': [255, 0, 0], 'K_impur': [255, 0, 255], 'Sain': [0, 255, 0], 'Sain_impur': [0, 0, 255], 'Trash':[150, 150, 150]}
colors = [class2color[concept2classes[conc]] for conc in concepts]
colors = np.array(colors, dtype=np.uint8)

In [ ]:
colors

array([[255,   0,   0],
       [255,   0, 255],
       [255,   0, 255],
       [  0, 255,   0],
       [  0, 255,   0],
       [  0, 255,   0],
       [  0,   0, 255],
       [150, 150, 150]], dtype=uint8)

In [ ]:
# export
for grade_fold in args.img_data.iterdir():
    if not grade_fold.is_dir():
        continue
    grade = grade_fold.name
    for scan_fold in grade_fold.iterdir():
        if not scan_fold.is_dir():
            continue
        scan_name = scan_fold.name
        print(scan_name)
        sub_df = df.loc[df['Slidename'] == scan_name]
        features = sub_df.values[:, 3:].astype(np.float32)
        result = np.argmin(((features[None] - centroids[:, None])**2).sum(-1), axis=0)
        
        segment_csv = args.seg_data/'Segmentation'/grade/f'{scan_name}.csv'
        scan_df = pd.read_csv(segment_csv, usecols=(0, 1, 2))
        segment = np.zeros((scan_df['YPosition'].max() // 299 + 1, scan_df['XPosition'].max() // 299 + 1, 3), dtype=np.uint8)
        for _, row in scan_df.iterrows():
            segment[row.YPosition//299, row.XPosition//299] = colors[result[row.Label]]
        outfile = args.outfolder/grade/f'{scan_name}.npy'
        if not outfile.parent.is_dir():
            outfile.parent.mkdir(parents=True)
        np.save(outfile, segment)
        segment = Image.fromarray(segment)
        segment.save(outfile.with_suffix('.png'))

CF_PACS04rescan_08P0124
CF_PACS04rescan_06P1204
CF_PACS04rescan_07P0211
CF_PACS04rescan_07P0252
CF_PACS04rescan_06P157
CF_PACS04rescan_06P0519
CF_PACS04rescan_06P0859
CF_PACS05HE_04074-133197
CF_PACS05HE_08009-A02.3597III1
CF_PACS05HE_12043-372424-8A
CF_PACS05HE_01002-779160
CF_PACS05HE_18010-04P06509
CF_PACS05HE_15008-33373
CF_PACS08rescan_364-09C2531
CF_PACS08rescan_329-11C2113
CF_PACS04rescan_06P200
CF_PACS04rescan_06P282
CF_PACS04rescan_06P0976
CF_PACS04rescan_06P0830
CF_PACS04rescan_07P0018
CF_PACS05HE_30017-04.4143.2C
CF_PACS05HE_02006-03.1861.02.01
CF_PACS05HE_01029-200400
CF_PACS05HE_07006-GG189T
CF_PACS05HE_02008-03.3186.02.02
CF_PACS08rescan_021-08C5067
CF_PACS08rescan_397-10C0646
CF_PACS08rescan_414-10C0998
CF_PACS08rescan_228-09C3830
CF_PACS08rescan_431-10C1066


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script(fname='_view_seg.ipynb')

Converted _view_seg.ipynb.
